### Install dependancies 

In [1]:
import numpy
import matplotlib.pyplot as plt
import pandas
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error 
from keras_tqdm import TQDMNotebookCallback


# fix random seed for reproducibility
numpy.random.seed(7)

DATA_DIR = 'historical_data'
FILE_NAME = "USDT_BTC.csv"

# load the dataset
dataframe = pandas.read_csv(DATA_DIR+"/"+FILE_NAME)

interval = dataframe['date'][1]-dataframe['date'][0] #interval in seconds
interval = interval/60 
#print(interval)

del dataframe['quoteVolume']
del dataframe['date']


Using TensorFlow backend.


In [2]:
col_names = dataframe.columns.values
print(col_names)

full_dataset = dataframe.values
full_dataset = full_dataset.astype('float32')

#pick out last column
dataset = np.ones([full_dataset.shape[0],1])
dataset[:,0] = full_dataset[:,5]

#normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

dataset = dataset[int(len(dataset)*0.333333):len(dataset)]

#split into train and test sets
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

['close' 'high' 'low' 'open' 'volume' 'weightedAverage']
99343 48931


In [3]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1, look_foward=1): 
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])    
    
    #for i in range(len(dataset)-look_back-look_foward-1):
    #    x = dataset[i:(i+look_back), 0]
    #    dataX.append(x)
    #    y = dataset[(i+look_back):(i+look_back+look_foward),0]
    #    dataY.append(y)
    return numpy.array(dataX), numpy.array(dataY)

look_back = 100
look_forward = 1
trainX, trainY = create_dataset(train, look_back, look_forward)
testX, testY = create_dataset(test, look_back, look_forward)

# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [4]:

model = Sequential()
model.add(LSTM(100, input_shape=(1, look_back)))
model.add(Dense(look_forward))
model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (None, 100)           80400       lstm_input_1[0][0]               
____________________________________________________________________________________________________
dense_1 (Dense)                  (None, 1)             101         lstm_1[0][0]                     
Total params: 80501
____________________________________________________________________________________________________


In [5]:
model.fit(trainX, trainY, nb_epoch=50, batch_size=1000, verbose=0, callbacks=[TQDMNotebookCallback()])

In [6]:
# make predictions
trainPredict = model.predict(trainX)
testPredict = model.predict(testX)

# invert predictions
trainPredict = scaler.inverse_transform(trainPredict)
trainY = scaler.inverse_transform(trainY)
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform(testY)


/home/dan/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:374: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/dan/anaconda3/lib/python3.5/site-packages/sklearn/preprocessing/data.py:374: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [7]:
# calculate root mean squared error at different levels of extrapolation and print results

print('           ', end='')
for i in range(look_forward):
    print('%4i  ' % (i+1), end='')

print('\nTrain RMSE: ', end='')
for i in range(look_forward):
    trainScore += np.sqrt(mean_squared_error(trainY[:,i], trainPredict[:,i]))
    print('%.1f ' % (trainScore), end='')
    
print('\nTest  RMSE: ', end='')    
for i in range(look_forward):
    testScore += np.sqrt(mean_squared_error(testY[:,i], testPredict[:,i]))
    print('%.1f ' % (trainScore), end='')



              1  
Train RMSE: 

NameError: name 'trainScore' is not defined

In [ ]:
print(train)

In [ ]:
%matplotlib inline

def make_plot(dataX, predY, title=''):
    # shift train predictions for plotting
    min2plt = look_back
    max2plt = 3*(N_pts/N_periods)
    rng = max2plt - min2plt
    lf = look_forward
    num2plt = int(rng/lf)

    data2plt = dataX[:,0,:][min2plt:max2plt,0]
    allpreds = predY[min2plt:max2plt,:]

    pred2plt = predY[min2plt-look_back:max2plt-look_back]
    pred2plt = np.zeros_like(data2plt)
    for i in range(num2plt):
        pred2plt[i*lf:(i+1)*lf] = allpreds[i,:]

    #plot baseline and predictions
    plt.plot()
    plt.plot(data2plt)
    plt.plot(pred2plt)
    plt.title(title)
    plt.show()

make_plot(trainX, trainPredict, title='test data')
make_plot(testX,  testPredict,  title='train data')
